In [0]:
pip install mlflow

In [0]:
# import relevant functions
import pandas as pd
from sklearn.model_selection import train_test_split
import boto3

In [0]:
s3 = boto3.client('s3')

In [0]:
# import data
bucket = "columbia-gr5069-main"
drivers_data = "raw/drivers.csv"
races_data = "raw/races.csv"
results_data = "raw/results.csv"

drivers = s3.get_object(Bucket= bucket, Key= drivers_data) 
races = s3.get_object(Bucket= bucket, Key= races_data) 
results = s3.get_object(Bucket= bucket, Key= results_data) 

df_drivers = pd.read_csv(drivers["Body"])
df_races = pd.read_csv(races["Body"])
df_results = pd.read_csv(results["Body"])

In [0]:
display(df_drivers)

In [0]:
display(df_races)

In [0]:
display(df_results)

In [0]:
# merge dataframes 
drivers_results = pd.merge(df_results, df_drivers, on="driverId", how="left")
drivers_results = pd.merge(drivers_results, df_races, on="raceId", how="left")

# calculate age of drivers at time of race
drivers_results['dob'] = pd.to_datetime(drivers_results['dob'])
drivers_results['date'] = pd.to_datetime(drivers_results['date'])
drivers_results['age'] = (drivers_results['date'].dt.year - drivers_results['dob'].dt.year) - ((
    drivers_results['date'].dt.year - drivers_results['dob'].dt.month) < 0)

# display final dataframe
display(drivers_results[["raceId", "driverId", "positionOrder", "nationality", "age"]])
